## data integration

In [1]:
import warnings
warnings.filterwarnings(action='ignore')
from functools import partial
from shapely import ops
import pyproj
import json, math
import matplotlib.pyplot as plt
from folium import plugins
import re

from matplotlib import font_manager, rc, rcParams
# def set_korea_font():
#     font_name = font_manager.FontProperties(fname="/System/Library/Fonts/Supplemental/AppleGothic.ttf").get_name()
#     rc('font', family=font_name)
#     rcParams.update({'font.size': 11})
#     rcParams['axes.unicode_minus'] = False  
# set_korea_font()

from geopy.distance import great_circle as distance
from geopy.point import Point as Point
from shapely.geometry import Point as shapely_Point
from math import sin, cos, atan2, sqrt, degrees, radians, pi

from geopy.distance import geodesic
import folium
from folium import plugins
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely
import os
import datetime
from tqdm import tqdm
#reproject import 필요
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
os.chdir('./data')

def reproject(geom, from_proj=None, to_proj=None):
    tfm = partial(pyproj.transform, pyproj.Proj(init=from_proj), pyproj.Proj(init=to_proj))
    return ops.transform(tfm, geom)

### 홍익선
1. 과속단속 카메라 개수
2. 유동인구 평균
3. 법규위반
4. 과속방지턱
5. 불법주정차
    - cctv 1개당 평균 불법주정차 단속 건수
    
### 조민주
1. 차량등록현황
2. 어린이 거주인구
	- 어린이 거주 비율
3. 주변 학교/어린이집 개수
4. 스쿨존 여부
5. 스쿨존 중복 포함 개수

### 심재훈
1. 사고 개수
2. 혼잡빈도
3. 너비
4. 교차로

### 이재훈
1. 학원 개수
2. 횡단보도 갯수
3. 신호등 갯수
4. 인도 여부

### data import

In [2]:
#tab에서 찾기 편하게 모든 단어에 _count_ 붙음
#refer_data 는 data_set['function name']
#홍익선
overspeed_cam_count_ = pd.read_csv('overspeed_cam.csv')
floating_pop_count_ = pd.read_csv('floating_pop.csv')
child_pedestrian_count_ = pd.read_csv('child_pedestrian.csv')
bump_count_ = pd.read_csv('bump.csv')
parking_count_ = pd.read_csv('parking.csv')
parking_cctv_count_ = [parking_count_, pd.read_csv('parking_cctv.csv')]

#조민주
car_count_ = gpd.read_file("3.오산시_차량등록현황_격자.geojson", encoding = "utf-8")
child_count_ = gpd.read_file("child_count.geojson", encoding = "utf-8")
child_rate_count_ = gpd.read_file("child_rate.geojson", encoding = "utf-8")
elem_kinder_count_ = gpd.read_file("elem_kinder_count.csv", encoding = "utf-8")
isSchoolZone_count_ = pd.read_csv("9.오산시_어린이보호구역(re).csv", encoding = "utf-8")
numberSchoolZone_count_ = isSchoolZone_count_

#심재훈
accident_count_ = pd.read_csv("accident_count.csv")
chaos1_nearby_count_ = [gpd.read_file('23.오산시_상세도로망_LV6.geojson'),
                 pd.read_csv('chaos1.csv')]
width_nearby_count_ = gpd.read_file('23.오산시_상세도로망_LV6.geojson')
cross_road_nearby_count_ = width_nearby_count_

#이재훈
num_cram_school_count_ = pd.read_csv("학원.csv")
shortest_cross_count_ = gpd.read_file("횡단보도.json",encoding="utf-8")
shortest_traffic_signal_count_ = gpd.read_file('19.오산시_신호등.geojson',encoding="utf-8")
shortest_sidewalk_count_ =  gpd.read_file("인도.json",encoding="utf-8")


data_set = {
'overspeed_cam_count': overspeed_cam_count_,
'floating_pop_count':floating_pop_count_,
'child_pedestrian_count':child_pedestrian_count_,
'bump_count':bump_count_,
'parking_count':parking_count_,
'parking_cctv_count':parking_cctv_count_,
'car_count':car_count_,
'child_count':child_count_,
'child_rate_count':child_rate_count_,
'elem_kinder_count':elem_kinder_count_,
'isSchoolZone_count':isSchoolZone_count_,
'numberSchoolZone_count':numberSchoolZone_count_,
'accident_count':accident_count_,
'chaos1_nearby_count':chaos1_nearby_count_,
'width_nearby_count':width_nearby_count_,
'cross_road_nearby_count':cross_road_nearby_count_,
'num_cram_school_count':num_cram_school_count_,
'shortest_cross_count':shortest_cross_count_,
'shortest_traffic_signal_count':shortest_traffic_signal_count_,
'shortest_sidewalk_count':shortest_sidewalk_count_,
}

In [3]:
#tuple이 str으로 저장되는 문제
def _str_to_tuple(string):
  """
  input: multilinestring object
  output: arr_cords; lon, lat
  ex) multiline_to_arr_cords(road_gdf.loc[0].geometry)
  """
  return re.findall(r'\d+\.\d+',string)

## 홍익선

## 1. 과속단속 카메라 개수

In [4]:
def overspeed_cam_count(lat, lon, radi, refer_data, idx=False):
  """
  input: origin lat lon, radi(m)
  output: following point count
  optional: (idx of point, count)
  """
  if (type(refer_data['geometry'][0]) == str):
    refer_data['geometry'] = refer_data['geometry'].apply(_str_to_tuple)
  #apply
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)
  if idx:
    return refer_data[bool_mask].index, sum(bool_mask)
  else:
    return sum(bool_mask)

## 2. 유동인구 평균

In [5]:
def floating_pop_count(lat, lon, radi, refer_data, idx=False):
  """
  input: origin lat lon, radi(km)
  output: following point count
  optional: (idx of point, count)
  """
  if (type(refer_data['geometry'][0]) == str):
    refer_data['geometry'] = refer_data['geometry'].apply(_str_to_tuple)
  #apply
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon), x).m <= radi)
  if idx:
    return refer_data[bool_mask].index, refer_data['mean'][bool_mask].mean()
  else:
    return refer_data['mean'][bool_mask].mean()

## 3. 법규위반

In [6]:
#어린이 보행/자전거 전체 사고 횟수
def child_pedestrian_count(lat, lon, radi, refer_data, typ = 'all', idx=False):
  """
  input: origin lat lon, radi(m), typ
  typ = 'all', '안전운전불이행','보행자보호의무위반', '신호위반'
  
  output: following point count
  optional: (idx of point, count)
  """
  if (type(refer_data['geometry'][0]) == str):
    refer_data['geometry'] = refer_data['geometry'].apply(_str_to_tuple)
  #apply
  if typ != 'all':
    refer_data = refer_data.loc[refer_data.법규위반==typ]
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)
  if idx:
    return refer_data[bool_mask].index, sum(bool_mask)
  else:
    return sum(bool_mask)

## 4. 과속방지턱

In [7]:
def bump_count(lat, lon, radi, refer_data, idx = False):
    if (type(refer_data['geometry'][0]) == str):
        refer_data['geometry'] = refer_data['geometry'].apply(_str_to_tuple)
    dist = refer_data[['index','geometry']].apply(lambda x: (geodesic((lat, lon), x[1]).m, x[0]), axis = 1)
    dist = min(dist, key = lambda x :x[0])
                                        
    if idx:
        return dist
    else:
        return dist[0]

## 5. 불법주정차

In [8]:
def parking_count(lat, lon, radi, refer_data, idx=False):
  """
  input: origin lat lon, radi(km)
  output: following point count
  optional: (idx of point, count)
  """
  if (type(refer_data['geometry'][0]) == str):
    refer_data['geometry'] = refer_data['geometry'].apply(_str_to_tuple)
  #apply
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)
  if idx:
    return refer_data[bool_mask].index, sum(bool_mask)
  else:
    return sum(bool_mask)

## 5-1. cctv 1개당 평균 불법주정차 단속 건수

In [9]:
def parking_cctv_count(lat, lon, radi, refer_data, idx=False):
  """
  cctv 1개당 평균 불법주정차 단속 건수
  input: origin lat lon, radi(km)
  output: following point count
  optional: (idx of point, count)
  """
  refer_data1, refer_data2 = refer_data[0], refer_data[1]
  if (type(refer_data1['geometry'][0]) == str):
    refer_data1['geometry'] = refer_data1['geometry'].apply(_str_to_tuple)
  if (type(refer_data2['geometry'][0]) == str):
    refer_data2['geometry'] = refer_data2['geometry'].apply(_str_to_tuple)
    
    
  #apply
  bool_mask1 = refer_data1['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)
  bool_mask2 = refer_data2['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)

  try:    
    result = sum(bool_mask1)/sum(bool_mask2)
  except:
    result = np.nan
  if idx:
    return refer_data1[bool_mask1].index, refer_data2[bool_mask2].index, result
  else:
    return result

## 조민주

In [10]:
osan = (37.1623799231016, 127.05436890115905)

In [11]:
#from https://medium.com/analytics-vidhya/calculating-distances-from-points-to-polygon-borders-in-python-a-paris-example-3b597e1ea291
def midpoint(a, b):
    a_lat, a_lon = radians(a.latitude), radians(a.longitude)
    b_lat, b_lon = radians(b.latitude), radians(b.longitude)
    delta_lon = b_lon - a_lon
    B_x = cos(b_lat) * cos(delta_lon)
    B_y = cos(b_lat) * sin(delta_lon)
    mid_lat = atan2(
        sin(a_lat) + sin(b_lat),
        sqrt(((cos(a_lat) + B_x) ** 2 + B_y ** 2))
    )
    mid_lon = a_lon + atan2(B_y, cos(a_lat) + B_x)
    # Normalise
    mid_lon = (mid_lon + 3 * pi) % (2 * pi) - pi

    return Point(latitude=degrees(mid_lat), longitude=degrees(mid_lon))

def get_line_midpoint(line):
    a = Point(line[0])
    b = Point(line[1])

    return midpoint(a,b)


def calculate_dist_to_line(line_a_lat, line_a_lng, line_b_lat, line_b_lng, point_object):
    a = Point(latitude=line_a_lat, longitude=line_a_lng)
    b = Point(latitude=line_b_lat, longitude=line_b_lng)
    dist = distance(midpoint(a, b), point_object)
    return dist


def get_min_distance_to_arr(arr_coords, point_object, unit='m'):
    min_dist = 999999
    line=[]
    for i, _ in enumerate(arr_coords):
        if i + 1 < len(arr_coords):
            dist = calculate_dist_to_line(
                line_a_lat=arr_coords[i][1],
                line_a_lng=arr_coords[i][0],
                line_b_lat=arr_coords[i + 1][1],
                line_b_lng=arr_coords[i + 1][0],
                point_object=point_object
            )
            if dist < min_dist:
                min_dist = dist
                line = [(arr_coords[i][1], arr_coords[i][0]), (arr_coords[i + 1][1], arr_coords[i + 1][0])]
        else:
            dist = calculate_dist_to_line(
                line_a_lat=arr_coords[i][1],
                line_a_lng=arr_coords[i][0],
                line_b_lat=arr_coords[0][1],
                line_b_lng=arr_coords[0][0],
                point_object=point_object
            )
            if dist < min_dist:
                min_dist = dist
                line = [(arr_coords[i][1], arr_coords[i][0]), (arr_coords[0][1], arr_coords[0][0])]

    if unit == 'm':
        return min_dist.m, line
    elif unit == 'km':
        return min_dist.km, line
    else:
        return min_dist, line
    
#additional interrial function
def _multiline_to_arr_cords(mls):
  """
  input: multilinestring object
  output: arr_cords; lon, lat
  ex) multiline_to_arr_cords(road_gdf.loc[0].geometry)
  """
  arr_cords = []
  lon_lat = re.findall(r'\d+\.\d+',str(mls))
  for i in range(0,len(lon_lat),2):
    arr_cords.append((float(lon_lat[i]),float(lon_lat[i+1])))
  return arr_cords

#(lat, lon), (lon, lat) 자꾸 갈려서 나오네
def _shPoint_to_list(shpoint):
  c1, c2 = re.findall(r'\d+\.\d+',str(shpoint))
  if c1 < c2:
    lon, lat = float(c1), float(c2)
  else:
    lat, lon = float(c1), float(c2)
  return lat, lon

def _list_to_gpPoint(mylist):
  return Point(latitude=mylist[0], longitude=mylist[1])

def _shPoint_to_gpPoint(shpoint):
  mylist = _shPoint_to_list(shpoint)
  return Point(latitude=mylist[0], longitude=mylist[1])

In [12]:
def road_nearby(lat, lon, radi, data):
  """
  input: origin lat lon, radi(m), data(shapely multilinestring)
  change in type: shapely(Euclidian) -> list -> geopy(Great Circle)
  #watchout for (lat lon), (lon lat)
  output: following roads
  
  #test_set
  lat, lon = (37.14875860564099, 127.0773701360968)
  radi = 1
  road_nearby(lat, lon, radi, road_gdf)
  """
  road_gdf = data
  dist_array = road_gdf.geometry.apply(lambda x: get_min_distance_to_arr(_multiline_to_arr_cords(x), _shPoint_to_gpPoint((lat, lon)), unit='m')[0])
  return road_gdf.loc[dist_array < radi,:]

## 2. 차랑등록현황

In [13]:
def car_count(lat, lon, radi, refer_data):
    """
    default unit = 'm'
    """   
    distance_arr = refer_data.geometry.apply(lambda x : get_min_distance_to_arr
                    (_multiline_to_arr_cords(x),
                     _shPoint_to_gpPoint((lat, lon)), unit = 'm')[0])
    sum_count = sum(refer_data[distance_arr <= radi]["car_cnt"])
    return sum_count

## 3. 어린이 거주인구 

In [14]:
def child_count(lat, lon, radi, refer_data):
    refer_data.val[refer_data.val.isna()] = 0
    distance_arr = refer_data.geometry.apply(lambda x : get_min_distance_to_arr
                    (_multiline_to_arr_cords(x),
                     _shPoint_to_gpPoint((lat, lon)), unit = 'm')[0])
    sum_count = sum(refer_data[distance_arr <= radi]["val"])
    return sum_count

## 3.5 어린이 거주 비율 

In [15]:
def child_rate_count(lat, lon, radi, refer_data):
    distance_arr = refer_data.geometry.apply(lambda x : get_min_distance_to_arr
                    (_multiline_to_arr_cords(x),
                     _shPoint_to_gpPoint((lat, lon)), unit = 'm')[0])
    selected_data = refer_data[distance_arr <= radi]
    try:
        child_rate = sum(selected_data["val"])/sum(selected_data["total"])
    except:
        child_rate = np.nan
    return child_rate

## 4. 주변 학교/어린이집 개수

In [16]:
def elem_kinder_count(lat, lon, radi, refer_data, typ = 'all'):
    """
    typ = 'all', '유치원과 어린이집', '초등학교'
    """
    if typ != 'all':
        refer_data = refer_data.loc[refer_data.시설구분==typ]
    data_loc = pd.Series(list(zip(refer_data["시설위치_위도"], refer_data["시설위치_경도"])))
    count_sum = data_loc.apply(lambda x: geodesic((lat, lon),x).m <= radi).sum()
    return count_sum

## 5. 스쿨존 여부 : isSchoolZone(lat, lon, radi, refer_data)
### 하지만 정확한 스쿨존 범위를 모름

In [17]:
def isSchoolZone_count(lat, lon, radi, refer_data): # radi = 300 이지만 순서 통일을 위해 default값 지정 x
    data_loc = pd.Series(list(zip(refer_data["보호구역_위도"], refer_data["보호구역_경도"])))
    count_sum = data_loc.apply(lambda x: geodesic((lat, lon),x).m <= 300).sum()
    return count_sum > 0

## 5.5 몇 개의 스쿨존에 포함되는지 : numberSchoolZone(lat, lon, radi, refer_data)
### CCTV 배치에 도움이 될 수도..?

In [18]:
def numberSchoolZone_count(lat, lon, radi, refer_data): # radi = 300 이지만 순서 통일을 위해 default값 지정 x
    data_loc = pd.Series(list(zip(refer_data["보호구역_위도"], refer_data["보호구역_경도"])))
    count_sum = data_loc.apply(lambda x: geodesic((lat, lon),x).m <= 300).sum()
    return count_sum

## 심재훈

### accident_count

In [19]:
def accident_count(lat, lon, radi, refer_data, idx=False):
  """
  input: origin lat lon, radi(m), data(accidents record)
  output: following point count
  optional: (idx of point, count)
  """

  if (type(refer_data['geometry'][0]) == str):
    refer_data['geometry'] = refer_data['geometry'].apply(_str_to_tuple)
  #apply
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon),x).m <= radi)
  if idx:
    return refer_data[bool_mask].index, sum(bool_mask)
  else:
    return sum(bool_mask)

In [20]:
def chaos1_nearby_count(lat, lon, radi, refer_data):
  """
  input: origin lat lon, radi(m), road_data(shapely multilinestring), chaos1_data
  output: following road's chaos1 by different times or lanes
  
  #test_set
  lat, lon = (37.14875860564099, 127.0773701360968)
  radi = 1
  chaos1_nearby(lat, lon, radi, road_gdf, chaos1_gdf)[11]
  #link_id 끝자리 01(상행) 02(하행) 순으로 나옴
  평일 주말 총 평균값
   """
  road_data, chaos1_data = refer_data[0], refer_data[1]
  roads = road_nearby(lat, lon, radi, road_data)
  result = roads.link_id.apply(lambda x: chaos1_data.loc[(chaos1_data.loc[:,'link_id']//100)==int(x),'chaos1'].mean())
  return result

def width_nearby_count(lat, lon, radi, road_data):
  roads = road_nearby(lat, lon, radi, road_data)
  return roads.width

def cross_road_nearby_count(lat, lon, radi, road_data):
  """
  4	교차로 통로
  32	복합교차로
  64	로타리
  128	회전교차로

  output = # of cross roads
  """
  cross_roads = [4,32,64,128]
  roads = road_nearby(lat, lon, radi, road_data)
  result = roads.loc[roads.link_type.apply(lambda x: int(x) in cross_roads)]
  return len(result)

## 이재훈

### 주변 학원 개수

In [21]:
def num_cram_school_count(lat, lon, radi, refer_data):
    cram_school = pd.Series(list(zip(refer_data["시설위치_위도"], refer_data["시설위치_경도"])))
    return cram_school.apply(lambda x: geodesic((lat, lon),x).m <= radi).sum()

### 횡단보도 거리

In [22]:
#reproject 후 polygon 그대로
def shortest_cross_count(lat, lon, radi, refer_data, idx = False):
    polygon1 = reproject(shapely_Point(lon,lat), 'EPSG:4326', 'EPSG:26944')
        
    result = refer_data[["OBJECTID","geometry"]].apply(lambda x : (x[1].distance(polygon1), x[0]), axis = 1)
    result = min(result, key = lambda x :x[0])
    if idx:
        return result
    return result[0]

### 신호등 거리

In [23]:
def shortest_traffic_signal_count(lat, lon, radi, refer_data, idx = False):
    result = refer_data[["OBJECTID","geometry"]].apply(lambda x : (geodesic((lat, lon),(x[1].y,x[1].x)).m, x[0]), axis = 1)
    result = min(result, key = lambda x :x[0])
    if idx:
        return result
    return result[0]

### 인도 여부

In [24]:
def shortest_sidewalk_count(lat, lon, radi, refer_data):
    polygon1 = reproject(shapely_Point(lon,lat), 'EPSG:4326', 'EPSG:26944')
    result = refer_data["geometry"].apply(lambda x : x.distance(polygon1))
    result = min(result)
    return result

# 데이터 만들기

In [25]:
function_dict = {
    "overspeed_cam_count" : overspeed_cam_count,
    "floating_pop_count" : floating_pop_count,
    "child_pedestrian_count" : child_pedestrian_count,
    "bump_count" : bump_count,
    "parking_count" : parking_count,
    "parking_cctv_count" : parking_cctv_count,
    "car_count" : car_count,
    "child_count" : child_count,
    "child_rate_count" : child_rate_count,
    "elem_kinder_count" : elem_kinder_count,
    "isSchoolZone_count" : isSchoolZone_count,
    "numberSchoolZone_count" : numberSchoolZone_count,
    "accident_count" : accident_count,
    "chaos1_nearby_count" : chaos1_nearby_count,
    "width_nearby_count" : width_nearby_count,
    "cross_road_nearby_count" : cross_road_nearby_count,
    "num_cram_school_count" : num_cram_school_count,
    "shortest_cross_count" : shortest_cross_count,
    "shortest_traffic_signal_count" : shortest_traffic_signal_count,
    "shortest_sidewalk_count" : shortest_sidewalk_count}

radi_dict = {
    'overspeed_cam_count' : 100,
    'floating_pop_count' : [12.5, 25]
#    'child_pedestrian_count' : 50 빼야함
    'bump_count' : [12.5 + 15, 25 + 15]
#    'parking_count' : 50,
    'parking_cctv_count' : [12.5, 25],
    'car_count' : 1000, # max값
    'child_count' : 1000, # max값
#    'child_rate_count' : 50,
    'elem_kinder_count' : 400, 
#     'isSchoolZone_count' : 400,
    'numberSchoolZone_count' : 400,
    'accident_count' : [12.5, 25],
    'chaos1_nearby_count' : [12.5, 25],
    'width_nearby_count' : [12.5, 25],
    'cross_road_nearby_count' : [12.5, 25],
    'num_cram_school_count' : 400,
    'shortest_cross_count' : 0,
    '중앙분리대' : [12.5, 25],
    'shortest_traffic_signal_count' : 0,
    'shortest_sidewalk_count' : [12.5, 25],} # 도로 보도의 유무

In [88]:
df = pd.read_csv("learn_point25.csv")[0:2]

function_dict[key](df.iloc[1,1],df.iloc[1,0],radi_dict[key], data_set[key])

In [89]:
key = "chaos1_nearby_count"
# (lat, lon, radi, refer_data)
keys.append(key)
print(key)
tmp = df.progress_apply(lambda x: function_dict[key](x[1], x[0], radi_dict[key], data_set[key]),axis = 1)
tmp

chaos1_nearby_count


  0%|          | 0/2 [00:00<?, ?it/s]

,8,9,14,17,158,159,160,161,770
0,NaN,90.53,NaN,NaN,NaN,99.58,98.93,49.25,NaN
1,NaN,90.53,84.97,99.48,39.93,99.58,98.93,49.25,99.805


# 바꾸면 되는 부분
이재훈 0 - 2214(exclud)  
홍익선 2214 - 4428(exclud)
심재훈 4428 - 6642(exclud)
조민주 6642 - 8856(exclud)

In [82]:
df = pd.read_csv("learn_point25.csv")[0:5]#2214#[6642:8856]#[4428:6642]#[2214:4428] 

In [83]:
values = []
keys = []
for key in list(data_set.keys())[13:14]:
    keys.append(key)
    print(key)
    tmp = df.progress_apply(lambda x: function_dict[key](x[1], x[0], radi_dict[key], data_set[key]),axis = 1)
    if len(tmp.shape) == 1:
        tmp.name = key
    else:
        tmp.columns = [key + str(i) for i in range(tmp.shape[1])]
    values.append(tmp)
values

chaos1_nearby_count


  0%|          | 0/5 [00:00<?, ?it/s]

[   chaos1_nearby_count0  chaos1_nearby_count1  chaos1_nearby_count2  \
 0                   NaN                 90.53                   NaN   
 1                   NaN                 90.53                   NaN   
 2                   NaN                 90.53                   NaN   
 3                   NaN                 90.53                 99.92   
 4                   NaN                 90.53                 99.92   
 
    chaos1_nearby_count3  chaos1_nearby_count4  chaos1_nearby_count5  \
 0                   NaN                   NaN                   NaN   
 1                 84.97                 99.48                   NaN   
 2                 84.97                 99.48                 52.07   
 3                   NaN                   NaN                   NaN   
 4                   NaN                   NaN                   NaN   
 
    chaos1_nearby_count6  chaos1_nearby_count7  chaos1_nearby_count8  \
 0                   NaN                   NaN              

In [76]:
# key = 'overspeed_cam_count'
values = []
keys = []
for key in data_set.keys():
    keys.append(key)
    print(key)
    tmp = df.progress_apply(lambda x: function_dict[key](x[1], x[0], radi_dict[key], data_set[key]),axis = 1)
    if len(tmp.shape) == 1:
        tmp.name = key
    else:
        tmp.columns = [key + str(i) for i in range(tmp.shape[1])]
    values.append(tmp)
    

overspeed_cam_count


  0%|          | 0/3 [00:00<?, ?it/s]

floating_pop_count


  0%|          | 0/3 [00:00<?, ?it/s]

child_pedestrian_count


...

  0%|          | 0/3 [00:00<?, ?it/s]

shortest_traffic_signal_count


  0%|          | 0/3 [00:00<?, ?it/s]

shortest_sidewalk_count


  0%|          | 0/3 [00:00<?, ?it/s]

In [78]:
values

[0    0
 1    0
 2    0
 Name: overspeed_cam_count, dtype: int64,
 0    390.898125
 1    507.739167
 2    364.718889
 Name: floating_pop_count, dtype: float64,
 0    1
 1    1
 2    1
 Name: child_pedestrian_count, dtype: int64,
 0    143.582568
 1    117.121977
 2     98.525988
 Name: bump_count, dtype: float64,
 0    0
 1    0
 2    5
 Name: parking_count, dtype: int64,
 0   NaN
 1   NaN
 2   NaN
 Name: parking_cctv_count, dtype: float64,
 0    2
 1    2
 2    2
 Name: car_count, dtype: int64,
 0    0.0
 1    0.0
 2    0.0
 Name: child_count, dtype: float64,
 0   NaN
 1   NaN
 2   NaN
 Name: child_rate_count, dtype: float64,
 0    0
 1    0
 2    0
 Name: elem_kinder_count, dtype: int64,
 0    False
 1    False
 2    False
 Name: isSchoolZone_count, dtype: bool,
 0    0
 1    0
 2    0
 Name: numberSchoolZone_count, dtype: int64,
 0    4
 1    5
 2    4
 Name: accident_count, dtype: int64,
    chaos1_nearby_count0  chaos1_nearby_count1  chaos1_nearby_count2  \
 0                   Na

In [77]:
pd.concat(values,axis=1)

,overspeed_cam_count,floating_pop_count,child_pedestrian_count,bump_count,parking_count,parking_cctv_count,car_count,child_count,child_rate_count,elem_kinder_count,...,width_nearby_count6,width_nearby_count7,width_nearby_count8,width_nearby_count9,width_nearby_count10,cross_road_nearby_count,num_cram_school_count,shortest_cross_count,shortest_traffic_signal_count,shortest_sidewalk_count
0,0,390.898125,1,143.582568,0,NaN,2,0.0,NaN,0,...,NaN,3,3,2,NaN,5,0,0.000000,4.965855,58.860463
1,0,507.739167,1,117.121977,0,NaN,2,0.0,NaN,0,...,3,3,3,2,4,8,0,19.934105,16.786558,42.782539
2,0,364.718889,1,98.525988,5,NaN,2,0.0,NaN,0,...,3,3,3,2,NaN,7,0,16.631819,21.477201,33.348896


In [30]:
print(keys[13], keys[14])


chaos1_nearby_count width_nearby_count


In [51]:
values[0].name = 1
values[0]

0    0
Name: 1, dtype: int64

In [72]:
a = values[14]
keys[13]


a

,8,9,159,160,161
0,1,1,3,3,2


In [44]:
a = values[0].to_frame().to_frame()


AttributeError: 'DataFrame' object has no attribute 'to_frame'

2. 사고를 막는 요인(ex 과속단속 카메라 개수)들의 경우, 사고가 많을수록 카메라가 많을 가능성이 높은데 이경우 어떻게 방지할 것인가.
-> 빼고 분석하자
3. prediction, 베타값을 활용한 최적화 방안을 구체적으로 어떻게 진행할 것인지. prediction의 경우 test데이터에 train데이터가 포함된다는 것을 막을수 있는 방안이 필요. 베타값활용의 경우 2번 문제뿐만 아니라 구체적인 fancy한 해결방안
-> 전체데이터로 분석, 어린이 보호구역 나눠서 분석, 도로특성 클러스터링 분석 => 최적화